In [1]:
# read data into pandas DataFrame
import pandas as pd

df = pd.read_csv('source/yelp_labelled.csv', header=None)
df.head(5)

,0,1,2,3,4,5
0,Wow... Loved this place.,1,NaN,NaN,NaN,NaN
1,Crust is not good.,0,NaN,NaN,NaN,NaN
2,Not tasty and the texture was just nasty.,0,NaN,NaN,NaN,NaN
3,Stopped by during the late May bank holiday of...,1,NaN,NaN,NaN,NaN
4,The selection on the menu was great and so wer...,1,NaN,NaN,NaN,NaN


In [2]:
df.shape

(992, 6)

## Data Preparation

In [3]:
# fill NA values
df.fillna(value='', inplace=True)
df.head(5)

,0,1,2,3,4,5
0,Wow... Loved this place.,1,,,,
1,Crust is not good.,0,,,,
2,Not tasty and the texture was just nasty.,0,,,,
3,Stopped by during the late May bank holiday of...,1,,,,
4,The selection on the menu was great and so wer...,1,,,,


In [4]:
# combine all columns to create 'review' column
df = df.astype('string')
df['review'] = df[0] + df[1] + df[2] + df[3] + df[4] + df[5]
df.review

0                              Wow... Loved this place.1
1                                    Crust is not good.0
2             Not tasty and the texture was just nasty.0
3      Stopped by during the late May bank holiday of...
4      The selection on the menu was great and so wer...
                             ...                        
987    I think food should have flavor and texture an...
988                            Appetite instantly gone.0
989    Overall I was not impressed and would not go b...
990    The whole experience was underwhelming and I t...
991    Then as if I hadn't wasted enough of my life t...
Name: review, Length: 992, dtype: string

In [5]:
# create 'sentiment' column using slice of 'review' column
df['sentiment'] = df.review.str.slice(start=-1)
df.sentiment

0      1
1      0
2      0
3      1
4      1
      ..
987    0
988    0
989    0
990    0
991    0
Name: sentiment, Length: 992, dtype: string

In [6]:
# remove score from 'review' column using slice of 'review' column
df.review = df.review.str.slice(stop=-1)
df.review

0                               Wow... Loved this place.
1                                     Crust is not good.
2              Not tasty and the texture was just nasty.
3      Stopped by during the late May bank holiday of...
4      The selection on the menu was great and so wer...
                             ...                        
987    I think food should have flavor and texture an...
988                             Appetite instantly gone.
989    Overall I was not impressed and would not go b...
990    The whole experience was underwhelming and I t...
991    Then as if I hadn't wasted enough of my life t...
Name: review, Length: 992, dtype: string

In [7]:
# update DataFrame to contain 'review' and 'sentiment' only
df = df[['review', 'sentiment']]
df.head(5)

,review,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


### Data Cleaning

In [8]:
# convert 'review' column to lowercase
df.review = df.review.str.lower()
df.review

0                               wow... loved this place.
1                                     crust is not good.
2              not tasty and the texture was just nasty.
3      stopped by during the late may bank holiday of...
4      the selection on the menu was great and so wer...
                             ...                        
987    i think food should have flavor and texture an...
988                             appetite instantly gone.
989    overall i was not impressed and would not go b...
990    the whole experience was underwhelming and i t...
991    then as if i hadn't wasted enough of my life t...
Name: review, Length: 992, dtype: string

In [9]:
# install nltk
#!pip install nltk

In [10]:
# import nltk for text pre-processing
# tokenize each row in 'review'
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
df.review = df.review.apply(tokenizer.tokenize)
df.review

0                              [wow, loved, this, place]
1                                 [crust, is, not, good]
2      [not, tasty, and, the, texture, was, just, nasty]
3      [stopped, by, during, the, late, may, bank, ho...
4      [the, selection, on, the, menu, was, great, an...
                             ...                        
987    [i, think, food, should, have, flavor, and, te...
988                          [appetite, instantly, gone]
989    [overall, i, was, not, impressed, and, would, ...
990    [the, whole, experience, was, underwhelming, a...
991    [then, as, if, i, hadn, t, wasted, enough, of,...
Name: review, Length: 992, dtype: object

In [11]:
# download stopwords corpus
# remove stopwords from each row in 'review'

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words = list(stop_words)

# remove select words from stop_words array
remove_words = ['isn', "isn't", 
                'not', 'nor', 'no', 
                'ain', 'aren', "aren't", 
                'couldn', "couldn't", 
                'didn', "didn't", 'doesn', "doesn't", 
                'haven', "haven't", 'hadn', "hadn't", 'hasn', "hasn't",
                'mightn', "mightn't", 'mustn', "mustn't", 
                'shouldn', "shouldn't", 'shan', "shan't",
                'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

for word in stop_words:
    if word in remove_words:
        stop_words.remove(word)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shantel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# function removes stopwords from each row in 'review'
def remove_stops(review):
    filtered_words = []
    for word in review:
        if word not in stop_words:
            filtered_words.append(word)
    return filtered_words

In [13]:
# remove stop_words
df.review = df.review.apply(remove_stops)
df.review

0                                    [wow, loved, place]
1                                     [crust, not, good]
2                           [not, tasty, texture, nasty]
3      [stopped, late, may, bank, holiday, rick, stev...
4                       [selection, menu, great, prices]
                             ...                        
987              [think, food, flavor, texture, lacking]
988                          [appetite, instantly, gone]
989      [overall, not, impressed, would, not, go, back]
990    [whole, experience, underwhelming, think, go, ...
991    [hadn, wasted, enough, life, poured, salt, wou...
Name: review, Length: 992, dtype: object

In [14]:
# lemmatize words using nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
 
wnl = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Shantel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
def get_lemms(review):
    lemms = []
    for word in review:
        lemms.append(wnl.lemmatize(word))
    return lemms

In [16]:
# lemmatize words
df.review = df.review.apply(get_lemms)
df.review

0                                    [wow, loved, place]
1                                     [crust, not, good]
2                           [not, tasty, texture, nasty]
3      [stopped, late, may, bank, holiday, rick, stev...
4                        [selection, menu, great, price]
                             ...                        
987              [think, food, flavor, texture, lacking]
988                          [appetite, instantly, gone]
989      [overall, not, impressed, would, not, go, back]
990    [whole, experience, underwhelming, think, go, ...
991    [hadn, wasted, enough, life, poured, salt, wou...
Name: review, Length: 992, dtype: object

In [17]:
# join string elements in 'review' column
# required for vectorization
df.review = df.review.str.join(' ')
df.review

0                                        wow loved place
1                                         crust not good
2                                not tasty texture nasty
3      stopped late may bank holiday rick steve recom...
4                             selection menu great price
                             ...                        
987                    think food flavor texture lacking
988                              appetite instantly gone
989              overall not impressed would not go back
990    whole experience underwhelming think go ninja ...
991    hadn wasted enough life poured salt wound draw...
Name: review, Length: 992, dtype: object

### Train/Test Split

In [18]:
# split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.review, df.sentiment, test_size=0.3, random_state=42)

Train/test output

In [19]:
# save train/test splits as CSV files
X_train.to_csv('output/x_train.csv', index=0)
X_test.to_csv('output/x_test.csv', index=0)
y_train.to_csv('output/y_train.csv', index=0)
y_test.to_csv('output/y_test.csv', index=0)

### Word Embeddings

In [20]:
# install tensorflow and keras
#!pip install tensorflow

In [21]:
# import tensorflow for
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [22]:
# use the text vectorization layer to normalize and map strings to integers
vectorize_layer = TextVectorization(
    standardize=None,
    output_mode='int',
    output_sequence_length=32)

In [23]:
# adapt train dataset to build the vocabulary.
vectorize_layer.adapt(X_train)

In [24]:
max_features = len(vectorize_layer.get_vocabulary())
print('Vocab size: ' + str(max_features))

Vocab size: 1477


### Model

In [25]:
# import layers and losses from tensorflow to create model
from tensorflow.keras import layers
from tensorflow.keras import losses

In [26]:
model = tf.keras.Sequential([
    layers.Embedding(max_features + 1, 8),
    layers.LSTM(8,dropout=0.5),
    layers.Dense(4), 
    layers.Dropout(0.5),
    layers.Dense(1)])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 8)           11824     
                                                                 
 lstm (LSTM)                 (None, 8)                 544       
                                                                 
 dense (Dense)               (None, 4)                 36        
                                                                 
 dropout (Dropout)           (None, 4)                 0         
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 12409 (48.47 KB)
Trainable params: 12409 (48.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Loss Function and Optimizer

In [27]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

### Training

In [28]:
history = model.fit(
    x=vectorize_layer(X_train),
    y=y_train.astype('int32'),
    validation_split=0.3,
    epochs=100)

Epoch 1/100


C:\Users\Shantel\.conda\envs\myenv\lib\site-packages\keras\src\engine\data_adapter.py:1798: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


16/16 [==============================] - 3s 48ms/step - loss: 0.6941 - binary_accuracy: 0.5134 - val_loss: 0.6937 - val_binary_accuracy: 0.4928
Epoch 2/100
16/16 [==============================] - 0s 15ms/step - loss: 0.6951 - binary_accuracy: 0.5010 - val_loss: 0.6937 - val_binary_accuracy: 0.4928
Epoch 3/100
16/16 [==============================] - 0s 15ms/step - loss: 0.6931 - binary_accuracy: 0.4784 - val_loss: 0.6932 - val_binary_accuracy: 0.4928
Epoch 4/100
16/16 [==============================] - 0s 14ms/step - loss: 0.6930 - binary_accuracy: 0.4742 - val_loss: 0.6931 - val_binary_accuracy: 0.5072
Epoch 5/100
16/16 [==============================] - 0s 13ms/step - loss: 0.6940 - binary_accuracy: 0.4825 - val_loss: 0.6931 - val_binary_accuracy: 0.5072
Epoch 6/100
16/16 [==============================] - 0s 13ms/step - loss: 0.6922 - binary_accuracy: 0.5155 - val_loss: 0.6930 - val_binary_accuracy: 0.5072
Epoch 7/100
16/16 [==============================] - 0s 11ms/step - loss: 0.